In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load your dataset
df = pd.read_csv('Sales_Data.csv')

In [3]:
# Assumptions
ordering_cost = 200  # ₹ per order
holding_cost_rate = 0.15  # 15% of unit price per year
Z = 1.65  # 95% service level

In [4]:
# Step 1: Aggregate product-level data
inventory_summary = df.groupby('product_ID').agg({
    'product_name': 'first',
    'price': 'mean',
    'demand_quantity': 'sum',
    'order_lead_time': 'mean'
}).reset_index()

In [5]:
# Step 2: Holding cost per unit
inventory_summary['holding_cost'] = inventory_summary['price'] * holding_cost_rate


In [6]:
# Step 3: EOQ Calculation
inventory_summary['EOQ'] = np.sqrt(
    (2 * inventory_summary['demand_quantity'] * ordering_cost) / inventory_summary['holding_cost']
)


In [7]:
# Step 4: Estimate daily demand stats
daily_demand_stats = df.groupby('product_ID')['demand_quantity'].agg(['mean', 'std']).rename(
    columns={'mean': 'daily_avg_demand', 'std': 'demand_std'}
)


In [8]:
# Step 5: Merge and handle missing std
inventory_summary = inventory_summary.merge(daily_demand_stats, on='product_ID')
median_std = inventory_summary['demand_std'].median()
inventory_summary['demand_std'].fillna(median_std, inplace=True)
inventory_summary['demand_std'].replace(0, median_std, inplace=True)


C:\Users\Groot\AppData\Local\Temp\ipykernel_11212\2227255563.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  inventory_summary['demand_std'].fillna(median_std, inplace=True)
C:\Users\Groot\AppData\Local\Temp\ipykernel_11212\2227255563.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [9]:
# Step 6: Compute Safety Stock and ROP
inventory_summary['Safety_Stock'] = Z * inventory_summary['demand_std'] * np.sqrt(inventory_summary['order_lead_time'])
inventory_summary['ROP'] = (
    inventory_summary['daily_avg_demand'] * inventory_summary['order_lead_time']
    + inventory_summary['Safety_Stock']
)


In [10]:
# Step 7: Finalize rule table
inventory_rules = inventory_summary[[
    'product_ID', 'product_name', 'EOQ', 'ROP', 'Safety_Stock'
]]

In [ ]:
# Save as CSV
inventory_rules.to_csv('inventory_rules.csv', index=False)